In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor # Try a multi-layer perceptron algorithm.

SETUP

In [2]:
#open directory with the spectra, PCA-reduced audio data, and register data in it.
os.chdir('./Training_data/')

SpectrumFeatures = pd.read_csv('000SampleSpectra.csv', index_col = 0) # Trying dBs for now.
#print(SpectrumFeatures.head())

WavFeatures = pd.read_csv('000PcaReducedWav.csv', index_col = 0)
#print(WavFeatures.head())

# Should combine SpectrumFeatures with WavFeatures to make one big feature array.
Features = SpectrumFeatures.join(WavFeatures)
print(Features.head())

RegisterTargets = pd.read_csv('000SampleRegisters.csv', index_col = 0)
RegisterTargets = RegisterTargets.drop(['Ratio1', 'Attack1', 'Decay1', 'Sustain1',
                        'Ratio2', 'Attack2', 'Decay2', 'Sustain2',
                        'Ratio3', 'Attack3', 'Decay3', 'Sustain3',
                        'Ratio4', 'Attack4', 'Decay4', 'Sustain4'], axis = 'columns') # Not needed here.
print(RegisterTargets.head())

# Below, we see the feature and target arrays.

                              HalfFF        1FF        2FF        3FF  \
01 - Circle of Life_0.wav  27.345405  30.462234 -19.134841 -25.059827   
01 - Circle of Life_1.wav -10.175424  20.070666  10.026839  11.563651   
01 - Circle of Life_2.wav -57.000795 -39.625398 -61.329705 -53.079755   
01 - Circle of Life_3.wav -13.490103  31.465241 -11.517152  -8.075195   
01 - Circle of Life_4.wav -13.608143  26.469035  17.800095   6.472096   

                                 4FF        5FF        6FF        7FF  \
01 - Circle of Life_0.wav -27.301782 -28.422371 -29.965447 -31.425729   
01 - Circle of Life_1.wav   5.924881  -0.834964  -4.774746  -6.389716   
01 - Circle of Life_2.wav -53.079755 -53.079755 -53.079755 -53.079755   
01 - Circle of Life_3.wav -13.541514 -23.127384 -28.912905 -30.748610   
01 - Circle of Life_4.wav   9.908368  -0.080632 -15.172361  -4.412612   

                                 8FF        9FF  ...            54  \
01 - Circle of Life_0.wav -33.067284 -33.085120  ...

TRAINING AN ESTIMATOR MODEL

In [3]:
# random state is a seed for reproducibility.
# can also try 'sgd' (stochastic gradient descent) or 'adam' (also SG-based)
# will try 'lbfgs' (a quasi-Newton method) for now.

RegressorMLP = MLPRegressor(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(100, 100, 100, 100), random_state=1)
RegressorMLP.fit(Features,RegisterTargets)

MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100), random_state=1,
             solver='lbfgs')

ANALYSIS

In [4]:
print(RegressorMLP.score(Features,RegisterTargets)) # 1.0 is best, 0 is on par with a constant model (predicts E[y]), and negative is bad.
print(RegressorMLP.get_params())

6.132660516977055e-16
{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100, 100, 100, 100), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'lbfgs', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


TESTING WITH A SAMPLE

In [5]:
# Need an audio sample; need to preprocess it with the PCA components already obtained.
# Then get its spectrum. Combine to get X.

# predict(X)
